In [ ]:
# Download Qwen2.5-Coder-7B-Instruct locally (works in Colab or local Jupyter)

import sys, subprocess, os, json, pathlib
def _pip(pkg): 
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", pkg], stdout=subprocess.PIPE)

try:
    from huggingface_hub import snapshot_download
except Exception:
    _pip("huggingface_hub>=0.23")
    from huggingface_hub import snapshot_download

MODEL_REPO = "Qwen/Qwen2.5-Coder-7B-Instruct"
LOCAL_DIR  = "./models/Qwen2.5-Coder-7B-Instruct"

# Optional: set a token if the repo ever requires auth (usually not for this one)
# os.environ["HUGGINGFACE_HUB_TOKEN"] = "<your_token_here>"

pathlib.Path(LOCAL_DIR).mkdir(parents=True, exist_ok=True)

local_path = snapshot_download(
    repo_id=MODEL_REPO,
    local_dir=LOCAL_DIR,
    local_dir_use_symlinks=False,   # store real files so it works fully offline
    allow_patterns=None,            # download all files in the repo
)

# Show a quick summary of what was downloaded
total_bytes = 0
files = []
for p in pathlib.Path(local_path).rglob("*"):
    if p.is_file():
        sz = p.stat().st_size
        total_bytes += sz
        files.append((str(p.relative_to(local_path)), sz))

files.sort()
print(f"✅ Downloaded to: {local_path}")
print(f"📦 Files: {len(files)} | Size: {total_bytes/1e9:.2f} GB\n")

# List the key files you should see
key_names = ["config.json","generation_config.json","tokenizer.json",
             "tokenizer_config.json","vocab.json","merges.txt",
             "model.safetensors.index.json"]
print("Key files present:")
for name in key_names:
    print(" -", name, "✓" if (pathlib.Path(local_path)/name).exists() else "⚠️ missing")

# (Optional) print the first few weight shards it found
print("\nWeight shards found:")
for f, sz in files:
    if f.endswith(".safetensors"):
        print(f" - {f}  ({sz/1e9:.2f} GB)")


c:\Users\jywu7\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:982: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

✅ Downloaded to: C:\Users\jywu7\OneDrive\Documents\MLE_capstone\app\model\models\Qwen2.5-Coder-7B-Instruct
📦 Files: 29 | Size: 15.24 GB

Key files present:
 - config.json ✓
 - generation_config.json ✓
 - tokenizer.json ✓
 - tokenizer_config.json ✓
 - vocab.json ✓
 - merges.txt ✓
 - model.safetensors.index.json ✓

Weight shards found:
 - model-00001-of-00004.safetensors  (4.88 GB)
 - model-00002-of-00004.safetensors  (4.93 GB)
 - model-00003-of-00004.safetensors  (4.33 GB)
 - model-00004-of-00004.safetensors  (1.09 GB)


In [5]:
from pathlib import Path
import os, sys

# --- Resolve repo root (works when notebook lives in app/model/) ---
cwd = Path.cwd()
repo_root = None
for cand in [cwd.parents[1], cwd, cwd.parent]:
    if (cand / "model").exists():
        repo_root = cand
        break
if repo_root is None:
    raise RuntimeError("Couldn't locate repo root that contains a 'model' folder.")

target_dir = (repo_root / "model" / "models" / "Qwen2.5-7B-Instruct-GPTQ").resolve()
target_dir.mkdir(parents=True, exist_ok=True)

# --- Choose a repo to pull from (change if you prefer another) ---
REPO_CANDIDATES = [
    "Qwen/Qwen2.5-7B-Instruct-GPTQ-Int4",  
]
REVISION = None  # or a tag/commit hash
HF_TOKEN = os.getenv("HF_TOKEN", None)  # set if the repo is gated

# --- Ensure huggingface_hub is available ---
try:
    from huggingface_hub import snapshot_download, login
except Exception:
    !{sys.executable} -m pip install -q huggingface_hub
    from huggingface_hub import snapshot_download, login

# (optional) faster transfers
os.environ.setdefault("HF_HUB_ENABLE_HF_TRANSFER", "1")

if HF_TOKEN:
    try:
        login(token=HF_TOKEN)
    except Exception as e:
        print("HF login failed (continuing unauthenticated):", e)

allow = [
    "*.safetensors",                   # includes model-00001-of-00002.safetensors
    "model.safetensors.index.json",
    "quantize_config.json",
    "config.json", "generation_config.json",
    "tokenizer.json", "tokenizer_config.json", "tokenizer.model",
    "special_tokens_map.json", "vocab.json",
]

last_err = None
for rid in REPO_CANDIDATES:
    print(f"→ trying {rid} …")
    try:
        local_path = snapshot_download(
            repo_id=rid,
            revision=REVISION,
            local_dir=str(target_dir),
            local_dir_use_symlinks=False,
            allow_patterns=allow,
            resume_download=True,
        )
        print("Downloaded to:", local_path)
        break
    except Exception as e:
        last_err = e
        print(f"  (failed: {e})")
else:
    raise RuntimeError(f"All candidates failed. Last error:\n{last_err}")

has_quant = (target_dir / "quantize_config.json").exists()
num_safetensors = len(glob.glob(str(target_dir / "*.safetensors")))
print("quantize_config.json:", has_quant)
print("# of safetensors weights:", num_safetensors)
assert has_quant and num_safetensors > 0, "This folder is not a valid GPTQ export."

print("\nSet this before running the console agent:")
print(rf'$env:HF_LOCAL_PATH = "{target_dir}"')


→ trying Qwen/Qwen2.5-7B-Instruct-GPTQ-Int4 …


Fetching 8 files:  25%|██▌       | 2/8 [02:33<07:39, 76.61s/it]


KeyboardInterrupt: 